In [1]:
import numpy as np
from flask import Flask, jsonify, request
from scipy.stats import beta

# create an app
app = Flask(__name__)

In [8]:
# define bandits
# there's no "pull arm" here
# since that's technically now the user/client
class Bandit:
    def __init__(self, name):
        self.click = 0
        self.view = 0
        self.name = name

    def sample(self):
        # let's assume the prior would be Beta(1,1)
        a = 1 + self.click
        b = 1 + self.view - self.click
        return np.random.beta(a, b)

    def add_click(self):
        self.click += 1

    def add_view(self):
        self.view += 1

        # print some helpful stats
        if self.views % 50 == 0:
            print(f"{self.name}: clks={self.click}, views={self.view}")

In [9]:
# initialize bandits
banditA = Bandit('A')
banditB = Bandit('B')

In [1]:
@app.route('/get_ad')
def get_ad():
    if banditA.sample() > banditB.sample():
        ad = 'A'
        banditA.add_view()
    else:
        ad = 'B'
        banditB.add_view()
    return jsonify({'advertisement_id': ad})


@app.route('/click_ad', methods=['POST'])
def click_ad():
    result = 'OK'
    if request.form['advertisement_id'] == 'A':
        banditA.add_click()
    elif request.form['advertisement_id'] == 'B':
        banditB.add_click()
    else:
        result = 'Invalid Input.'

    # nothing to return really
    return jsonify({'result': result})


if __name__ == '__main__':
    app.run(host='127.0.0.1', port='1234')